## Configuração de Ambiente

In [1]:
from config import result_index

results_folder = f"results_{result_index}"
results_file = f"{results_folder}/finetuning_results.csv"

In [2]:
%pip install --upgrade matplotlib bertopic plotly datamapplot
%pip install openai
%pip install transformers sympy
%pip install wordcloud
%pip install -u kaleido
%pip install numpy==1.24.4 gensim

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\vmart\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\vmart\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\vmart\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



Usage:   
  C:\Users\vmart\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  C:\Users\vmart\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  C:\Users\vmart\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install [options] [-e] <vcs project url> ...
  C:\Users\vmart\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install [options] [-e] <local project path> ...
  C:\Users\vmart\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install [options] <archive url/path> ...

no such option: -u


   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.8 MB 653.6 kB/s eta 0:00:23
    --------------------------------------- 0.3/14.8 MB 2.8 MB/s eta 0:00:06
   ----- ---------------------------------- 2.0/14.8 MB 14.4 MB/s eta 0:00:01
   ----------- ---------------------------- 4.3/14.8 MB 23.2 MB/s eta 0:00:01
   ----------------- ---------------------- 6.3/14.8 MB 27.1 MB/s eta 0:00:01
   --------------------- ------------------ 7.9/14.8 MB 29.6 MB/s eta 0:00:01
   --------------------------- ------------ 10.2/14.8 MB 32.8 MB/s eta 0:00:01
   ----------------------------------- ---- 13.0/14.8 MB 50.4 MB/s eta 0:00:01
   ---------------------------------------  14.8/14.8 MB 46.7 MB/s eta 0:00:01
   ---------------------------------------- 14.8/14.8 MB 40.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4


  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\vmart\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import os
import numpy as np

from gensim.corpora import Dictionary

from tqdm import tqdm
import gc
import logging
from itertools import product

from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance, KeyBERTInspired, OpenAI
from sentence_transformers import SentenceTransformer
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering, KMeans

from hdbscan import HDBSCAN
import openai

C:\Users\vmart\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Importação de Dados

In [4]:
# Substitua pelo caminho correto
file_path = f'processed_data_{result_index}/cleaned_videos.csv'

df = pd.read_csv(file_path)

In [5]:
df.sample(5)

,Unnamed: 0.1,Unnamed: 0,video_id,title,description,channel_id,published_at,category_id,tags,view_count,...,concurrent_viewers,active_live_chat_id,recording_date,topicCategories,processing_status,parts_total,parts_processed,time_left_ms,processing_failure_reason,text
264,264,294.0,nO06xfLips8,Mãe sensata? É ótimo! Mas homem que só amadure...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-12 21:00:54,22,[],113,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Society'],NaN,0,0,0,NaN,mãe sensata é ótimo ma homem que só amadurece ...
349,832,NaN,wUQwbUKrzME,Não seja o cara que implora! Mulher admira cor...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-06 21:01:05,22,[],129,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0,0,0,NaN,não seja cara que implora mulher admira corage...
228,228,255.0,w7tH6YIZyqc,MULHER DIZ QUE PAGA MARIDO PARA TER RELAÇÃO CO...,"Olá! seja bem vindo a mais um video do canal, ...",UCRmNflJuD1TxLbRlDV08_7g,2025-05-15 01:55:09,24,"['red pill', 'divorcio', 'pensão socio afetiva...",2718,...,0,NaN,NaN,"['https://en.wikipedia.org/wiki/Humour', 'http...",NaN,0,0,0,NaN,mulher diz que paga marido para ter relação co...
20,20,21.0,2YhQqPj6aSQ,Homem feio tem mais chance de ser fiel? | Qual...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-29 21:00:02,22,[],708951,...,0,NaN,NaN,[],NaN,0,0,0,NaN,homem feio tem mais chance de ser fiel qual su...
314,794,NaN,Fdz39IZljTg,COROA CONTA QUE CONHECEU UM NOVINHO NUMA REDE ...,seja bem vindo a mais um vídeo do canal Não se...,UCUBeVY6Kn7ulBmUGynJqISw,2025-05-09 02:19:06,24,"['redpill', 'msol', 'miqueinha', 'relacionamen...",1148,...,0,NaN,NaN,"['https://en.wikipedia.org/wiki/Humour', 'http...",NaN,0,0,0,NaN,coroa conta que conheceu um novinho numa rede ...


In [6]:
print(len(df), " textos")

df.columns

447  textos


Index(['Unnamed: 0.1', 'Unnamed: 0', 'video_id', 'title', 'description',
       'channel_id', 'published_at', 'category_id', 'tags', 'view_count',
       'like_count', 'comment_count', 'duration', 'definition', 'caption',
       'licensed_content', 'privacy_status', 'license', 'embeddable',
       'public_stats_viewable', 'is_made_for_kids', 'thumbnail_url',
       'default_audio_language', 'default_language', 'actual_start_time',
       'scheduled_start_time', 'actual_end_time', 'scheduled_end_time',
       'concurrent_viewers', 'active_live_chat_id', 'recording_date',
       'topicCategories', 'processing_status', 'parts_total',
       'parts_processed', 'time_left_ms', 'processing_failure_reason', 'text'],
      dtype='object')

In [8]:
df['channel_id'].unique()

array(['UCO9FRrBUwGdYopkMbGGKbpg', 'UCAYoI16-UkXemcnhC-kTvDQ',
       'UCRmNflJuD1TxLbRlDV08_7g', 'UCUBeVY6Kn7ulBmUGynJqISw',
       'UCeL1a4rpEA8UG9IQIewPccg', 'UC3nQ4xUl6rodOWuQbBULyow',
       'UCNiU1wZxK6YN-KuJP7QMpBQ', 'UCX0VSzJ2z5l0C9wnwh5SoRw',
       'UCExFA9MsrRmWnXUlhiwu4qA'], dtype=object)

In [9]:
train_videos = df.sample(n=100, random_state=42)

In [10]:
train_data = pd.concat([train_videos])

In [11]:
train_data.sample(5)

,Unnamed: 0.1,Unnamed: 0,video_id,title,description,channel_id,published_at,category_id,tags,view_count,...,concurrent_viewers,active_live_chat_id,recording_date,topicCategories,processing_status,parts_total,parts_processed,time_left_ms,processing_failure_reason,text
378,863,NaN,k_9RUZzvCwY,PRESAS DIZEM PARA AS NOVINHAS NÃO FAZER COISA ...,seja bem vindo a mais um vídeo do canal Não se...,UCUBeVY6Kn7ulBmUGynJqISw,2025-05-05 11:19:45,24,"['redpill', 'msol', 'miqueinha', 'relacionamen...",682,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0,0,0,NaN,presas dizem para novinhas não fazer coisa err...
75,75,80.0,9U9Rq6_B5Ts,Sinais que ela vai te colocar na cadeia! | Qua...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-26 15:00:25,22,[],37994,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Health'],NaN,0,0,0,NaN,sinai que ela vai te colocar na cadeia qual su...
79,79,84.0,7qkwWygX8y8,A mulher que não é submissa ao marido está fad...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-26 12:00:13,22,[],75205,...,0,NaN,NaN,[],NaN,0,0,0,NaN,mulher que não é submissa ao marido está fadad...
84,537,NaN,lRscpUdKfVU,Rapaz leva flores e amiga da namorada debocha....,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-25 15:01:12,22,[],8802,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Society'],NaN,0,0,0,NaN,rapaz lev flores e amiga da namorada debocha q...
66,517,NaN,GjYlzOd82Qc,MULHER GRAVANDO PARA PROVAR QUE NÃO PRECISA DE...,"Olá! seja bem vindo a mais um video do canal, ...",UCRmNflJuD1TxLbRlDV08_7g,2025-05-27 02:01:00,24,"['red pill', 'divorcio', 'pensão socio afetiva...",255,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Entertainment'...,NaN,0,0,0,NaN,mulher gravando para provar que não precisa de...


In [12]:
# Remove textos vazios ou contendo apenas espaços em branco
train_sentences = [text for text in train_data['text'].tolist() if isinstance(text, str) and text.strip()]

# Execução do Grid Search

os hiperparâmetros do modelo podem influenciar significativamente os resultados do modelo e uma escolha indevida poderia compremeter os resultados. Uma estratégia conhecida para evitar valores inadequados de hiperparâmetros é a Grid Search, que se trata de uma busca iterativa com parâmetros pré-selecionados. Os hiperparâmetros testados e a coerência gerada a partir de suas combinações estão visiveis na seção abaixo. A partir do melhor resultado encontrado a partir da busca extensiva, todos os dados foram integralmente submetidos a outra instância do algoritmo, usando-se os hiperpâmetros selecionados.

Apesar de diversos hiperparâmetros estarem sendo avaliados, destaco aqui os modelos de pré-treino de embedding e o uso do TF-IDF para e UMAP para aprimorar e reduzir a dimensionalidade dos dados.

In [13]:
param_grid = {
    "embedding_model_name": [
        "all-MiniLM-L6-v2",
        "roberta",
        "paraphrase-MiniLM-L6-v2"
    ],
    "reduction_model_name": [
        "umap"
    ],
    "umap_params": [
        {"n_neighbors": 5, "n_components": 5, "min_dist": 0.1},
        {"n_neighbors": 20, "n_components": 5, "min_dist": 0.1},
        {"n_neighbors": 50, "n_components": 5, "min_dist": 0.1},
        {"n_neighbors": 100, "n_components": 5, "min_dist": 0.1},
        {"n_neighbors": 5, "n_components": 5, "min_dist": 0.5},
        {"n_neighbors": 20, "n_components": 5, "min_dist": 0.5},
        {"n_neighbors": 50, "n_components": 5, "min_dist": 0.5},
        {"n_neighbors": 100, "n_components": 5, "min_dist": 0.5},
    ],  # Parâmetros para UMAP
    "clustering_model_name": [
        "hdbscan",
        # "agglomerative",
        "kmeans"
    ],
    "representation_model_name": [
        "maximal_marginal_relevance"
    ]
}

In [14]:
def get_embedding_model(name):
    """Retrieve embedding model based on name."""
    models = {
        "all-MiniLM-L6-v2": SentenceTransformer('all-MiniLM-L6-v2'),
        "roberta": SentenceTransformer('roberta-base-nli-stsb-mean-tokens'),
        "paraphrase-MiniLM-L6-v2": SentenceTransformer('paraphrase-MiniLM-L6-v2')
    }
    return models.get(name, None)

def get_reduction_model(reduction_model_name, umap_params=None):
    """
    Função para retornar o modelo de redução de dimensionalidade baseado no nome.
    """
    if reduction_model_name == "umap":
        import umap
        return umap.UMAP(
            n_neighbors=umap_params.get("n_neighbors", 15),
            n_components=umap_params.get("n_components", 2),
            min_dist=umap_params.get("min_dist", 0.1),
            random_state=42,
            low_memory=True
        )
    raise ValueError(f"Reduction model '{reduction_model_name}' não suportado.")

def get_clustering_model(name, min_cluster_size):
    """Retrieve clustering model based on name."""
    if name == "hdbscan":
        return HDBSCAN(min_cluster_size=min_cluster_size, prediction_data=True)
    elif name == "agglomerative":
        return AgglomerativeClustering(n_clusters=min_cluster_size)
    elif name == "kmeans":
        return KMeans(n_clusters=min_cluster_size, random_state=42)
    return None

def get_representation_model(name, diversity):
    """Retrieve representation model."""
    if name == "mmr":
        return MaximalMarginalRelevance(diversity=diversity)
    elif name == "keybert":
        return KeyBERTInspired(diversity=diversity)
    elif name == "openai":
        from config import OPENAI_KEY
        client = openai.OpenAI(api_key=OPENAI_KEY)
        return OpenAI(client, model="gpt-3.5-turbo", chat=True)
    return None

def calculate_coherence(topic_model, sentences):
    """Calculate coherence score for a BERTopic model."""
    texts = [sentence.split() for sentence in sentences]
    dictionary = Dictionary(texts)
    topic_ids = topic_model.get_topics().keys()
    topics = [[word for word, _ in topic_model.get_topics()[topic_id]] for topic_id in topic_ids]
    coherence_model = CoherenceModel(
        topics=topics,
        texts=texts,
        dictionary=dictionary,
        coherence="c_v"
    )
    return coherence_model.get_coherence()

In [15]:
# Configuração de logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def perform_grid_search(sentences, param_grid, results_file):
    """
    Realiza busca em grade para hiperparâmetros de BERTopic ou utiliza os melhores parâmetros salvos.
    """
    best_coherence = -np.inf

    # Calcula todas as combinações de parâmetros
    all_combinations = list(product(*param_grid.values()))
    total_combinations = len(all_combinations)

    try:
        results_df = pd.read_csv(results_file)
        if not results_df.empty:
            logging.info("\n=== Alguns modelos já foram previamente calculados ===")

            # Verifica se todas as combinações já foram calculadas
            calculated_combinations = results_df[[
                "embedding_model", "reduction_model", "umap_params",
                "clustering_model", "representation_model"
            ]].drop_duplicates().shape[0]

            if calculated_combinations == total_combinations:
                logging.info("Todas as combinações de parâmetros já foram calculadas.")
    except FileNotFoundError:
        results_df = pd.DataFrame(columns=[
            "embedding_model", "reduction_model", "umap_params",
            "clustering_model", "representation_model", "coherence"
        ])
    except Exception as e:
        logging.error(f"Erro ao carregar os melhores parâmetros: {e}")

    # Caso as combinações não estejam completas, realiza a busca em grade
    logging.info("\n=== Starting Grid Search ===")
    logging.info(f"Total parameter combinations: {total_combinations}")

    with tqdm(total=total_combinations, desc="Grid Search Progress", unit="combination") as pbar:
        if not results_df.empty:
            already_calculated = set(
                tuple(row) for row in results_df[[
                    "embedding_model", "reduction_model", "umap_params",
                    "clustering_model", "representation_model"
                ]].drop_duplicates().itertuples(index=False, name=None)
            )
        else:
            already_calculated = set()

        for params in all_combinations:
            param_key = {
                "embedding_model": params[0],
                "reduction_model": params[1],
                "umap_params": str(params[2]),
                "clustering_model": params[3],
                "representation_model": params[4]
            }

            if tuple(param_key.values()) in already_calculated:
                existing_coherence = results_df.loc[
                    (results_df[list(param_key)] == pd.Series(param_key)).all(axis=1), "coherence"
                ].values[0]
                logging.info(f"Modelo já calculado: {param_key} | Coherence: {existing_coherence}")
                pbar.update(1)
                continue

            try:
                logging.info(f"Calculando novos parâmetros: {param_key}")
                embedding_model = get_embedding_model(param_key["embedding_model"])
                reduction_model = get_reduction_model(param_key["reduction_model"], eval(param_key["umap_params"]))
                clustering_model = get_clustering_model(param_key["clustering_model"], 30)
                tfidf_vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
                representation_model = get_representation_model(param_key["representation_model"], None)

                logging.info("Initializing model...")
                topic_model = BERTopic(
                    n_gram_range=2,
                    language="english",
                    umap_model=reduction_model,
                    hdbscan_model=clustering_model,
                    vectorizer_model=tfidf_vectorizer,
                    representation_model=representation_model,
                    embedding_model=embedding_model,
                    min_topic_size=100,
                )

                logging.info("Fitting model...")
                topic_model.fit_transform(sentences)
            except Exception as e:
                logging.error(f"Erro com parâmetros {param_key}: {e}")
                pbar.update(1)
                continue

            logging.info("Calculating coherence...")
            coherence_score = calculate_coherence(topic_model, sentences)
            new_result = {**param_key, "coherence": coherence_score}
            results_df = pd.concat([results_df, pd.DataFrame([new_result])], ignore_index=True)
            results_df.to_csv(results_file, index=False)

            if coherence_score > best_coherence:
                best_coherence = coherence_score

            del embedding_model, reduction_model, clustering_model, tfidf_vectorizer, representation_model, topic_model
            gc.collect()

            pbar.update(1)

    return results_df

In [17]:
%pip install --upgrade sentence-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\vmart\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
perform_grid_search(train_sentences, param_grid, results_file)

2025-06-14 19:27:08,391 - INFO - 
=== Starting Grid Search ===
2025-06-14 19:27:08,391 - INFO - Total parameter combinations: 48
Grid Search Progress:   0%|          | 0/48 [00:00<?, ?combination/s]2025-06-14 19:27:08,393 - INFO - Calculando novos parâmetros: {'embedding_model': 'all-MiniLM-L6-v2', 'reduction_model': 'umap', 'umap_params': "{'n_neighbors': 5, 'n_components': 5, 'min_dist': 0.1}", 'clustering_model': 'hdbscan', 'representation_model': 'maximal_marginal_relevance'}
2025-06-14 19:27:08,397 - INFO - Use pytorch device_name: cpu
2025-06-14 19:27:08,398 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-06-14 19:27:18,471 - WARNING - No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.
2025-06-14 19:27:38,624 - ERROR - Erro com parâmetros {'embedding_model': 'all-MiniLM-L6-v2', 'reduction_model': 'umap', 'umap_params': "{'n_neighbors': 5, 'n_components': 5, 'min_dist': 0.1}", 'clustering

,embedding_model,reduction_model,umap_params,clustering_model,representation_model,coherence
0,roberta,umap,"{'n_neighbors': 5, 'n_components': 5, 'min_dis...",hdbscan,maximal_marginal_relevance,0.377603
1,roberta,umap,"{'n_neighbors': 5, 'n_components': 5, 'min_dis...",kmeans,maximal_marginal_relevance,0.509318
2,roberta,umap,"{'n_neighbors': 20, 'n_components': 5, 'min_di...",hdbscan,maximal_marginal_relevance,0.377603
3,roberta,umap,"{'n_neighbors': 20, 'n_components': 5, 'min_di...",kmeans,maximal_marginal_relevance,0.557656
4,roberta,umap,"{'n_neighbors': 50, 'n_components': 5, 'min_di...",hdbscan,maximal_marginal_relevance,0.377603
5,roberta,umap,"{'n_neighbors': 50, 'n_components': 5, 'min_di...",kmeans,maximal_marginal_relevance,0.550874
6,roberta,umap,"{'n_neighbors': 100, 'n_components': 5, 'min_d...",hdbscan,maximal_marginal_relevance,0.377603
7,roberta,umap,"{'n_neighbors': 100, 'n_components': 5, 'min_d...",kmeans,maximal_marginal_relevance,0.566453
8,roberta,umap,"{'n_neighbors': 5, 'n_components': 5, 'min_dis...",hdbscan,maximal_marginal_relevance,0.377603
9,roberta,umap,"{'n_neighbors': 5, 'n_components': 5, 'min_dis...",kmeans,maximal_marginal_relevance,0.554769


In [19]:
results_df = pd.read_csv(results_file)
results_df.sort_values(by="coherence", ascending=False)

,embedding_model,reduction_model,umap_params,clustering_model,representation_model,coherence
29,paraphrase-MiniLM-L6-v2,umap,"{'n_neighbors': 50, 'n_components': 5, 'min_di...",kmeans,maximal_marginal_relevance,0.592118
15,roberta,umap,"{'n_neighbors': 100, 'n_components': 5, 'min_d...",kmeans,maximal_marginal_relevance,0.566509
7,roberta,umap,"{'n_neighbors': 100, 'n_components': 5, 'min_d...",kmeans,maximal_marginal_relevance,0.566453
3,roberta,umap,"{'n_neighbors': 20, 'n_components': 5, 'min_di...",kmeans,maximal_marginal_relevance,0.557656
11,roberta,umap,"{'n_neighbors': 20, 'n_components': 5, 'min_di...",kmeans,maximal_marginal_relevance,0.557588
21,paraphrase-MiniLM-L6-v2,umap,"{'n_neighbors': 50, 'n_components': 5, 'min_di...",kmeans,maximal_marginal_relevance,0.555407
9,roberta,umap,"{'n_neighbors': 5, 'n_components': 5, 'min_dis...",kmeans,maximal_marginal_relevance,0.554769
5,roberta,umap,"{'n_neighbors': 50, 'n_components': 5, 'min_di...",kmeans,maximal_marginal_relevance,0.550874
13,roberta,umap,"{'n_neighbors': 50, 'n_components': 5, 'min_di...",kmeans,maximal_marginal_relevance,0.549375
19,paraphrase-MiniLM-L6-v2,umap,"{'n_neighbors': 20, 'n_components': 5, 'min_di...",kmeans,maximal_marginal_relevance,0.546947


In [20]:
df.sample(5)

,Unnamed: 0.1,Unnamed: 0,video_id,title,description,channel_id,published_at,category_id,tags,view_count,...,concurrent_viewers,active_live_chat_id,recording_date,topicCategories,processing_status,parts_total,parts_processed,time_left_ms,processing_failure_reason,text
310,790,NaN,Q3ukRP_TUR4,MULHER DIZENDO QUE NÃO É NORMAL HOMEM MAIS VEL...,"Olá! seja bem vindo a mais um video do canal, ...",UCRmNflJuD1TxLbRlDV08_7g,2025-05-09 12:04:47,24,"['red pill', 'divorcio', 'pensão socio afetiva...",654,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Humour'],NaN,0,0,0,NaN,mulher dizendo que não é normal homem mais vel...
11,11,12.0,1kIR_cBtHjM,esses homens estão PREFERINDO A SOLITUDE — e n...,💪 Use o cupom (CONSELHO) na Growth: https://ww...,UCX0VSzJ2z5l0C9wnwh5SoRw,2025-05-30 14:20:01,26,[],20138,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Society'],NaN,0,0,0,NaN,esses homens estão preferindo solitude e não é...
335,335,370.0,pPyX0NtHjLw,"Gritos, reclamações e raiva: o caminho mais rá...",#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-07 21:01:00,22,[],119,...,0,NaN,NaN,[],NaN,0,0,0,NaN,gritos reclamações e raiva caminho mais rápido...
425,425,466.0,MuScLmEv_ck,"Se encontrou um Homem de valor, valorize! Apre...",#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-02 15:00:41,22,[],358,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0,0,0,NaN,se encontrou um homem de valor valorize aprend...
228,228,255.0,w7tH6YIZyqc,MULHER DIZ QUE PAGA MARIDO PARA TER RELAÇÃO CO...,"Olá! seja bem vindo a mais um video do canal, ...",UCRmNflJuD1TxLbRlDV08_7g,2025-05-15 01:55:09,24,"['red pill', 'divorcio', 'pensão socio afetiva...",2718,...,0,NaN,NaN,"['https://en.wikipedia.org/wiki/Humour', 'http...",NaN,0,0,0,NaN,mulher diz que paga marido para ter relação co...


In [21]:
sentences = df['text']

In [22]:
import ast  # Substituir eval por ast.literal_eval para maior segurança
from utils.save_models_and_variables import load_model, load_variable
try:
    topic_model = load_model(f'{results_folder}/bertopic_{result_index}')
    print("Previously calculated" if topic_model else "Will be calculated...")

    topics = load_variable(f'{results_folder}/topics_bertopic_{result_index}')
    print("Topics previously calculated" if topics else "Topics will be calculated...")

    probs = load_variable(f'{results_folder}/probs_bertopic_{result_index}')
    print("Probs previously calculated" if probs is not None else "Probs will be calculated...")
except:
    topic_model = None
    topics = None
    probs = None

if topic_model is None or probs is None or topics is None:
    # sentences = unified_df["text"]
    
    # Passo 2: Validar colunas no DataFrame
    expected_columns = [
        "embedding_model",
        "reduction_model",
        "umap_params",
        "clustering_model",
        "representation_model",
        "coherence"  # Deve existir como métrica para identificar os melhores parâmetros
    ]

    missing_columns = [col for col in expected_columns if col not in results_df.columns]
    if missing_columns:
        raise ValueError(f"As colunas seguintes estão faltando no results_df: {missing_columns}")

    # Passo 3: Identificar os melhores parâmetros
    best_row = results_df.loc[results_df['coherence'].idxmax()]

    # Garantir que umap_params seja convertido corretamente
    try:
        best_umap_params = ast.literal_eval(best_row['umap_params'])
    except (ValueError, SyntaxError) as e:
        raise ValueError(f"Erro ao converter umap_params: {best_row['umap_params']}") from e

    # Extrair os melhores parâmetros
    best_params = {
        "embedding_model": best_row['embedding_model'],
        "reduction_model": best_row["reduction_model"],
        "umap_params": best_umap_params,
        "clustering_model": best_row['clustering_model'],
        "representation_model": best_row["representation_model"],
    }
    best_coherence = best_row["coherence"]

    print("Melhores parâmetros:", best_params)
    print("Melhor coerência:", best_coherence)

    # Continuar o restante do fluxo
    print("Getting embedding model...")
    embedding_model = get_embedding_model(best_params["embedding_model"])

    print("Getting reduction model...")
    reduction_model = get_reduction_model(best_params["reduction_model"], best_params["umap_params"])

    print("Getting clustering model...")
    clustering_model = get_clustering_model(best_params["clustering_model"], 30)

    print("Getting TFIDF model...")
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)

    print("Getting representation model...")
    representation_model = get_representation_model(best_params["representation_model"], None)
    
    print("Initializing model...")
    if topic_model is None:
        topic_model = BERTopic(
            language="portuguese",
            umap_model=reduction_model,
            hdbscan_model=clustering_model,
            vectorizer_model=tfidf_vectorizer,
            representation_model=representation_model,
            embedding_model=embedding_model,
            min_topic_size=100,
            nr_topics=5
        )
        
    print("Fitting model...")
    topics, probs = topic_model.fit_transform(sentences)
    print("Fitted")
    
    try:
        df["topic_id"] = topics
    except:
        topic_mapping = dict(zip(sentences, topics))

        # 🔹 4️⃣ Adicionar os tópicos ao DataFrame
        df['topic'] = df['text'].map(topic_mapping)
else:
    df = pd.read_csv(f"{results_folder}/topic_data.csv")
    print("Topic data previously calculated")

2025-06-15 11:37:36,349 - INFO - Use pytorch device_name: cpu
2025-06-15 11:37:36,350 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Will be calculated...
Failed to load variable from results_2025_06_10/topics_bertopic_2025_06_10.pkl: [Errno 2] No such file or directory: 'results_2025_06_10/topics_bertopic_2025_06_10.pkl'
Topics will be calculated...
Failed to load variable from results_2025_06_10/probs_bertopic_2025_06_10.pkl: [Errno 2] No such file or directory: 'results_2025_06_10/probs_bertopic_2025_06_10.pkl'
Probs will be calculated...
Melhores parâmetros: {'embedding_model': 'paraphrase-MiniLM-L6-v2', 'reduction_model': 'umap', 'umap_params': {'n_neighbors': 50, 'n_components': 5, 'min_dist': 0.5}, 'clustering_model': 'kmeans', 'representation_model': 'maximal_marginal_relevance'}
Melhor coerência: 0.5921175102080011
Getting embedding model...


2025-06-15 11:37:38,486 - INFO - Use pytorch device_name: cpu
2025-06-15 11:37:38,487 - INFO - Load pretrained SentenceTransformer: roberta-base-nli-stsb-mean-tokens
2025-06-15 11:37:40,740 - INFO - Use pytorch device_name: cpu
2025-06-15 11:37:40,742 - INFO - Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2


Getting reduction model...
Getting clustering model...
Getting TFIDF model...
Getting representation model...
Initializing model...
Fitting model...
Fitted


In [24]:
try:
    # Certifique-se de que o modelo está treinado antes de reduzir os outliers
    if not hasattr(topic_model, "topic_sizes_"):  # Verifica se já foi treinado
        topic_model.fit(sentences)  # Treina o modelo com as frases

    # Reduzindo outliers
    new_topics = topic_model.reduce_outliers(sentences, topics, strategy='c-tf-idf', threshold=0.05)

    # Criando o vetorizador TF-IDF (caso não tenha sido definido antes)
    vectorizer_model = TfidfVectorizer(stop_words='portuguese', smooth_idf=True)

    # Atualizando o modelo com os novos tópicos
    topic_model.update_topics(sentences, topics=new_topics, vectorizer_model=vectorizer_model)

    # Exibindo informações sobre os tópicos
    print(topic_model.get_topic_info())
    print("Outliers reduced")
except ValueError:
    print("No outliers to reduce.")

No outliers to reduce.


In [25]:
coherence_score = calculate_coherence(topic_model, sentences)

print(f"Esse modelo tem coerência de {coherence_score}")

2025-06-15 12:01:33,883 - INFO - adding document #0 to Dictionary<0 unique tokens: []>
2025-06-15 12:01:33,912 - INFO - built Dictionary<3981 unique tokens: ['###', '#conquistafeminina', '#mensagensdebomdia', '#psicologiamasculina', '#redpillfeminina']...> from 447 documents (total 52301 corpus positions)
2025-06-15 12:01:33,912 - INFO - Dictionary lifecycle event {'msg': "built Dictionary<3981 unique tokens: ['###', '#conquistafeminina', '#mensagensdebomdia', '#psicologiamasculina', '#redpillfeminina']...> from 447 documents (total 52301 corpus positions)", 'datetime': '2025-06-15T12:01:33.912759', 'gensim': '4.3.3', 'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'created'}
2025-06-15 12:01:33,914 - INFO - using ParallelWordOccurrenceAccumulator<processes=11, batch_size=64> to estimate probabilities from sliding windows
2025-06-15 12:01:44,879 - INFO - 1 batches submitted to accumulate sta

Esse modelo tem coerência de 0.8459217505426448


In [26]:
# Dicionário para armazenar os nomes dos tópicos
topic_names_dict = {}

# Obter informações dos tópicos
topics = topic_model.get_topic_info()

# Gerar títulos para os tópicos
for topic_id in topics['Topic']:
    top_words = [word for word, _ in topic_model.get_topic(topic_id)[:5]]
    topic_name = "_".join(top_words)
    topic_names_dict[topic_id] = topic_name
    
topic_names_dict[-1] = 'outlier'

# Exibir o dicionário resultante
topic_names_dict

{0: 'evitar_feito_antiotário_ser_redpillbrasil',
 1: 'memes_vindo_humor_canal_não',
 2: 'capítulo_como_homem_ele_emocional',
 3: 'http_redcast_master_junior_instagram',
 4: 'http_conselho_projetoconselhosubstackcom_comercial_recomendo',
 -1: 'outlier'}

In [27]:
print(f"{len(topic_names_dict)} topicos gerados")

6 topicos gerados


In [29]:
print(f"{len(df)} antes do filtro")
df = df.drop_duplicates(subset=[ "video_id"])
df = df.dropna(subset=["video_id"])
print(f"{len(df)} depois do filtro")

447 antes do filtro
447 depois do filtro


In [30]:
# 🔹 4️⃣ Comparação dentro de cada caso   
def print_topic(case):
    case_df = df[df["case"] == case]
    
    print(f"\n🔹 Comparação de tópicos para o caso: {case}")
    
    # Verificar quando o mesmo autor tem um tópico diferente para vídeo e notícia
    differing_topics = case_df[case_df["news_topic"] != case_df["video_topic"]]
    
    print(f"Número de diferenças entre tópicos no caso {case}: {len(differing_topics)}")

    # Se quiser ver exemplos específicos de diferenças:
    return differing_topics[['news_author', 'video_author', 'news_topic', 'video_topic', 'news_title', 'video_title', 'news_text', 'video_text']]

## Exportação de Dados

In [31]:
from utils.save_models_and_variables import save_model, save_variable

save_model(topic_model, f'{results_folder}/bertopic')
save_variable(topics, f'{results_folder}/topics_bertopic')
save_variable(probs, f'{results_folder}/probs_bertopic')
save_variable(topic_names_dict, f'{results_folder}/topics_dict')

Saved model: results_2025_06_10/bertopic.pkl
Saved variable: results_2025_06_10/topics_bertopic.pkl
Saved variable: results_2025_06_10/probs_bertopic.pkl
Saved variable: results_2025_06_10/topics_dict.pkl


In [32]:
df.to_csv(f'{results_folder}/topic_data.csv')

In [33]:
len(df)

447